# Importing and preparing supermarkets data

## Libraries and settings

In [2]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: /Users/adisaljusi/repos/data_analytics/Week_02
supermarkets.json


## Importing data

In [3]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [4]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [20]:
df1.tags[0].keys()

dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])

In [18]:
# Get all unique keys from all dictionaries in the tags column
all_keys = set()
for tags_dict in df1.tags:
    all_keys.update(tags_dict.keys())

# Convert to sorted list for better readability
all_keys = sorted(list(all_keys))
print(f"Total number of unique keys: {len(all_keys)}")
print("\nAll available keys:")
for key in all_keys:
    print(f"  {key}")

Total number of unique keys: 233

All available keys:
  access
  access:covid19
  addr:city
  addr:city:de
  addr:city:fr
  addr:country
  addr:floor
  addr:full
  addr:housename
  addr:housenumber
  addr:place
  addr:postcode
  addr:street
  addr:street:de
  addr:street:fr
  addr:street_1
  addr:suburb
  air_conditioning
  alt_name
  amenity
  atm
  branch
  brand
  brand:fr
  brand:wikidata
  brand:wikipedia
  building
  building:levels
  bulk_purchase
  cash_withdrawal
  cash_withdrawal:fee
  cash_withdrawal:operator
  cash_withdrawal:purchase_required
  cash_withdrawal:type
  changing_table
  changing_table:location
  check_date
  check_date:opening_hours
  clothes
  comment
  contact:addr
  contact:addr:full
  contact:city
  contact:email
  contact:facebook
  contact:fax
  contact:housenumber
  contact:instagram
  contact:phone
  contact:postcode
  contact:street
  contact:website
  created_by
  cuisine
  currency:CHF
  currency:EUR
  currency:Leman
  currency:USD
  delivery
  del

In [6]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2 = df2[['brand', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode', 'opening_hours']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode',})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,shop,city,street,housenumber,postcode,opening_hours
0,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
2,Coop,supermarket,Uznach,NaN,NaN,8730,NaN
3,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off


## Merge df1 and df2

In [7]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode', 'opening_hours']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730,NaN
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off


## Count and identify the number of missing values (if any)

In [8]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type                0
id                  0
lat                 0
lon                 0
brand            1065
shop                0
city             1777
street           1608
housenumber      1680
postcode         1709
opening_hours    1361
dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
5,node,60271452,47.406671,9.305450,NaN,supermarket,NaN,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,supermarket,NaN,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,supermarket,NaN,NaN,NaN,NaN,"Mo-Sa 08:00-12:00, 13:30-18:00"
13,node,95582038,47.050385,9.059214,NaN,supermarket,NaN,NaN,NaN,NaN,Mo-Su 09:00-21:00;PH off
...,...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,supermarket,NaN,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,supermarket,NaN,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 07:30-17:00
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN,Mo-Fr 07:30-19:00; Th 07:30-20:00; Sa 07:30-18:30


## Count and identify duplicated values (if any)

In [9]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours


## Get data types of all variables

In [10]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type              object
id                 int64
lat              float64
lon              float64
brand             object
shop              object
city              object
street            object
housenumber       object
postcode          object
opening_hours     object
dtype: object

### Save data to file

In [11]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Additional filters on supermarkets

#### Filter for Migros supermarkets in Zurich

In [12]:
df_filtered = df.loc[(df['shop'] == 'supermarket') & (df['brand'] == 'Migros')]
print(f"Shape: {df_filtered.shape}")
df_filtered.head(5)

Shape: (532, 11)


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
11,node,83330862,47.344749,8.529981,Migros,supermarket,Zürich,Etzelstrasse,3,8038,Mo-Sa 07:30-21:00; PH off
16,node,119249170,47.375255,8.536107,Migros,supermarket,Zürich,Löwenstrasse,31-35,8001,Mo-Sa 09:00-20:00; PH off


In [13]:
df_filtered = df.loc[
    (df['brand'] == 'Coop') &
    (df['shop'] == 'supermarket') &
    (df['city'].isin(['Zürich', 'Zurich', 'Basel', 'Bern']))
]

print(f"Total supermarkets: {df_filtered.shape[0]}")
print("Count by city:")
print(df_filtered['city'].value_counts())

df_filtered.head(5)

Total supermarkets: 52
Count by city:
city
Zürich    37
Basel     10
Bern       5
Name: count, dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
9,node,79977755,47.340070,8.530546,Coop,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038,Mo-Sa 07:30-21:00
59,node,265776668,47.376417,8.559594,Coop,supermarket,Zürich,Zürichbergstrasse,75,8044,Mo-Fr 07:30-21:00; Sa 07:30-20:00
63,node,266630559,47.377716,8.511219,Coop,supermarket,Zürich,Badenerstrasse,333,8003,Mo-Sa 07:00-22:00
70,node,267345511,47.385809,8.516574,Coop,supermarket,Zürich,Maagplatz,1,8005,Mo-Fr 06:00-20:00; Sa 08:00-20:00


In [14]:
df_filtered = df.loc[
    (df['shop'] == 'supermarket') &
    df['brand'].notna() &
    df['city'].notna() &
    df['housenumber'].notna() &
    df['postcode'].notna()
]
print(f"Shape: {df_filtered.shape}")
df_filtered.head(5)

Shape: (1148, 11)


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
8,node,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038,Mo-Sa 07:30-21:00


In [15]:
df_filtered = df.loc[
    (df['shop'] == 'supermarket') &
    df['opening_hours'].notna() 
]
print(f"Shape: {df_filtered.shape}")
df_filtered.head(5)

Shape: (2031, 11)


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [16]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 25.0.0
Datetime: 2025-10-09 09:43:49
Python Version: 3.11.13
-----------------------------------
